In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import cv2

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import warnings

warnings.simplefilter("ignore", UserWarning)
from skimage.io import imread, imshow
from skimage.transform import resize

In [33]:
excel_data = pd.read_csv("sentiment.csv")

In [34]:
del excel_data['Unnamed: 0']

# check and remove invalid file_name

In [35]:
invalid = []
file_names = excel_data["filename"]
dir = "C:\\Kazim 1\\Fast Uni\\Semester 4\\AI\\Project\\sentiment_images/"
for image in file_names:
    path = dir+image
    try:
        i = load_img(path, target_size=(60, 60))
    except:
        invalid.append(image)
        
set(invalid)

{'COCO_val2014_000000299631.jpg',
 'COCO_val2014_000000047010.jpg',
 'COCO_val2014_000000233825.jpg',
 'COCO_val2014_000000553097.jpg',
 'COCO_val2014_000000036238.jpg',
 'COCO_val2014_000000568358.jpg',
 'COCO_val2014_000000321674.jpg',
 'COCO_val2014_000000207913.jpg',
 'COCO_val2014_000000183988.jpg',
 'COCO_val2014_000000183675.jpg',
 'COCO_val2014_000000234794.jpg',
 'COCO_val2014_000000207785.jpg',
 'COCO_val2014_000000108669.jpg',
 'COCO_val2014_000000192651.jpg',
 'COCO_val2014_000000207728.jpg',
 'COCO_val2014_000000276076.jpg',
 'COCO_val2014_000000118260.jpg',
 'COCO_val2014_000000473774.jpg',
 'COCO_val2014_000000102400.jpg',
 'COCO_val2014_000000237864.jpg',
 'COCO_val2014_000000346138.jpg',
 'COCO_val2014_000000435986.jpg',
 'COCO_val2014_000000251816.jpg',
 'COCO_val2014_000000020069.jpg',
 'COCO_val2014_000000286021.jpg',
 'COCO_val2014_000000351711.jpg',
 'COCO_val2014_000000478474.jpg',
 'COCO_val2014_000000482305.jpg',
 'COCO_val2014_000000484229.jpg',
 'COCO_val2014

In [36]:
excel_data.drop(excel_data[excel_data.filename == 'COCO_val2014_000000130712.jpg'].index , inplace=True)
excel_data.drop(excel_data[excel_data.filename == 'COCO_val2014_000000310622.jpg'].index , inplace=True)
excel_data.drop(excel_data[excel_data.filename == 'COCO_val2014_000000421673.jpg'].index , inplace=True)
excel_data.drop(excel_data[excel_data.filename == 'COCO_val2014_000000359276.jpg'].index , inplace=True)

# analysis of data

analysis of labels 

In [37]:
excel_data.sentiment.value_counts()

1    21229
0    17880
Name: sentiment, dtype: int64

In [47]:
rows = int(len(image_tensors)*0.8)
train = excel_data.iloc[:rows]
test = excel_data.iloc[rows:]

In [49]:
s1 = train['sentiment']
s2 = test['sentiment']

In [53]:
print(s1.value_counts())
print(s2.value_counts())

1    16842
0    14445
Name: sentiment, dtype: int64
1    4387
0    3435
Name: sentiment, dtype: int64


difference between the 2 labels is not significant

# image feature extraction for unique images only

In [13]:
import os
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model

In [19]:
file_names = excel_data['filename']
files = file_names.unique()

In [20]:
model = VGG16()
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
#vgg16 is a a classifier so we extract features from 2nd last layer
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [23]:
features = {}
directory = 'C:\Kazim 1\Fast Uni\Semester 4\AI\Project\sentiment_images/'
for image in files:
    image_path = directory+image
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
    img = preprocess_input(img)
    feature = model.predict(img, verbose=0)
    features[image] = feature

In [27]:
import copy
deepcopied_dict = copy.deepcopy(features)

# Feature store and load

In [3]:
import pickle
# Write the dictionary to a file using pickle.dump
# with open('image.pkl', 'wb') as f:
#     pickle.dump(features, f)

In [4]:
# Load the dictionary back from the file using pickle.load
with open('image.pkl', 'rb') as f:
    loaded_dict_img = pickle.load(f)

In [80]:
# loaded_dict_img

# convert array to tensor for neural network


In [18]:
# def convert_to_tensor(file):
#     tensor = torch.from_numpy(loaded_dict_img[file][0])
#     return tensor

def convert_to_tensor(file):
    tensor = torch.from_numpy(file)
    return tensor

In [16]:
file_names = excel_data['filename']
image_features = []
for file in file_names:
    image_features.append(loaded_dict_img[file][0])

In [26]:
j = np.array(image_features)
image_tensors = convert_to_tensor(j)

In [30]:
with open('image_tensors.pkl', 'wb') as f:
    pickle.dump(image_tensors, f)

In [31]:
image_tensors

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 3.4262, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 3.4262, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 3.4262, 0.0000],
        ...,
        [5.9334, 4.3203, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [5.9334, 4.3203, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [5.9334, 4.3203, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

# unique tensors

In [87]:
unique_tensors = []
files = file_names.unique()
for file in files:
    unique_tensors.append(convert_to_tensor(file))

# copying features to correspondig rows

In [88]:
feat = []
files = excel_data["filename"]
for file in files:
    feat.append(convert_to_tensor(file))

# confirming size of features of all images

In [89]:
lenghts = []
for i in unique_tensors:
    lenghts.append(len(i))
set(lenghts) #hence proved all leghts are same 

{4096}

In [104]:
excel_data['tensors'] = feat

In [126]:
with open('image_tensors.pkl', 'wb') as f:
    pickle.dump(feat, f)